In [1]:
import requests

In [3]:
dtm_url = "https://www.opengeodata.nrw.de/produkte/geobasis/hm/dgm1_xyz/dgm1_xyz/"
r = requests.get(dtm_url)

In [18]:
from bs4 import BeautifulSoup 
import os

In [20]:
data_dir = r"../data/derived/DTM_NRW_EPSG_25832/"
out_fname = r"NRW_DTM_Tiles_BB.gpkg"


In [5]:
soup = BeautifulSoup(r.content, 'html.parser')

In [21]:
os.makedirs(data_dir,exist_ok = True)


In [6]:
tag_list = soup.find_all('file')

In [7]:
fname_list = []

for tag in tag_list:
    fname_list.append(tag["name"])

In [29]:
def points_from_fname(fname,Dx=1000,Dy=1000):
    x_L = int(fname.split("_")[2])*1000
    y_L = int(fname.split("_")[3])*1000
    x_R = x_L + Dx
    y_U = y_L + Dy
    
    P_LL = (x_L,y_L)
    P_UL = (x_L,y_U)
    P_UR = (x_R,y_U)
    P_LR = (x_R,y_L)
    
    return[P_LL,P_UL,P_UR,P_LR]

In [30]:
from shapely.geometry import Polygon 

In [31]:
import geopandas 
import geopandas as gpd

In [32]:
geom_list = [Polygon(points_from_fname(fname)) for fname in fname_list]

In [33]:
dic = {'fname': fname_list, 'geometry': geom_list}
gdf = gpd.GeoDataFrame(data = dic, crs="EPSG:25832")
gdf.head()

,fname,geometry
0,dgm1_32_280_5652_1_nw.xyz.gz,"POLYGON ((280000.000 5652000.000, 280000.000 5..."
1,dgm1_32_280_5653_1_nw.xyz.gz,"POLYGON ((280000.000 5653000.000, 280000.000 5..."
2,dgm1_32_280_5654_1_nw.xyz.gz,"POLYGON ((280000.000 5654000.000, 280000.000 5..."
3,dgm1_32_280_5655_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
4,dgm1_32_280_5656_1_nw.xyz.gz,"POLYGON ((280000.000 5656000.000, 280000.000 5..."


In [34]:
print(f"Write gdf to file {data_dir + out_fname:s}")
gdf.to_file(data_dir + out_fname, driver = "GPKG") 

Write gdf to file ../data/derived/DTM_NRW_EPSG_25832/NRW_DTM_Tiles_BB.gpkg


In [4]:
%%time
from osgeo import gdal
basename = "dgm1_32_317_5730_1_nw"
infile = basename + r".xyz.gz"
outfile = basename + r".tif"
print("In: ", infile)
ds = gdal.Open(infile)
ds = gdal.Translate(outfile, ds, outputSRS="EPSG:25832")
ds = None
print("Out: ", outfile)

In:  dgm1_32_317_5730_1_nw.xyz.gz
Out:  dgm1_32_317_5730_1_nw.tif
Wall time: 4.1 s
